In [1]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# The Web Crawler

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # Starts up chrome

#time.sleep()


""" Enters systembolaget.se and clicks a button confirming
age>20.

Afterwards it accesses the first page of all 'Red Wines' on the page and starts to scroll and scrape 
the dynamic website. The While loop  iterates sequentially over all pages and collects and creates the soup"""

soups=[]
driver.get("https://beta.systembolaget.se/sok/?categoryLevel1=Vin&categoryLevel2=R%C3%B6tt%20vin&page=1")
elem = driver.find_element_by_xpath('//button[@class="css-75g0v6"]')
elem.click()
elem=driver.find_element_by_xpath('//*')
page = 1
while page!=130:
    for i in range(18):
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN) # Use send_keys(Keys.HOME) to scroll up to the top of page
        time.sleep(0.2)
    
    
        elem=driver.find_element_by_xpath('//*')
        text=elem.get_attribute('innerHTML')
        soup=BeautifulSoup(text)
        elem.get_attribute('outerHTML')
    
    product_link = [i.get('href') for i in soup.select('a.css-1mgic9w')]
    soups.append(product_link)
    page+=1
    driver.get(f"https://beta.systembolaget.se/sok/?categoryLevel1=Vin&categoryLevel2=R%C3%B6tt%20vin&page={page}")
    
    
# Concatenating the all the collected links to every wine's page
base_link = 'https://beta.systembolaget.se'
links = [''.join(base_link + j) for i in soups for j in i]



# Goes on to the main site again and confirms age>20.


driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://beta.systembolaget.se/sok/?categoryLevel1=Vin&categoryLevel2=R%C3%B6tt%20vin&page=1")
elem = driver.find_element_by_xpath('//button[@class="css-75g0v6"]')
elem.click()
elem=driver.find_element_by_xpath('//*')


# Creating empty lists for all the future columns
namn=[]
land=[]
pris=[]
liter_pris=[]
reg_nr=[]
forpackning=[]
storlek=[]
alkoholhalt=[]
sockerhalt=[]
passar_till=[]
experiment = []
experiment1 = []
år = []




# Then the program iterates over all the links and scrolls to the bottom of the page, to refresh again.
# The loop retrieves information about the wines using css-selectors

for i in links:
    driver.get(i)
    for i in range(2):
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN) # Use send_keys(Keys.HOME) to scroll up to the top of page
        time.sleep(0.3)
        elem=driver.find_element_by_xpath('//*')
        text=elem.get_attribute('innerHTML')
        soup=BeautifulSoup(text)
        #elem.get_attribute('outerHTML')
    soup1 = [i.text for i in soup.select('h3.css-1lk5v66')]
    namn.append(soup1)
    
    soup2 = [i.text for i in soup.select('span.css-z4h1l3 strong')]
    land.append(soup2)
    
    soup3 = [i.text for i in soup.select('div.css-3ezmqi')]
    pris.append(soup3)
    
    soup4 = [i.text for i in soup.select('div.css-hob2m3 span')]
    liter_pris.append(soup4)
    
    soup5 = [i.text for i in soup.select('div.css-16d9hab > :nth-of-type(1)')]
    reg_nr.append(soup5)
    
    soup6 = [i.text for i in soup.select('div.css-16d9hab > :nth-of-type(2)')]
    forpackning.append(soup6)
    
    soup7 = [i.text for i in soup.select('div.css-16d9hab > :nth-of-type(2) span')]
    storlek.append(soup7)
    
    soup8 = [i.text for i in soup.select('div.css-16d9hab > :nth-of-type(3) span')]
    alkoholhalt.append(soup8)
    
    soup9 = [i.text for i in soup.select('div.css-16d9hab > :nth-of-type(4) span')]
    sockerhalt.append(soup9)
    
    soup10 = [i.text for i in soup.select('p.css-33dxyf')]
    passar_till.append(soup10)
    
    soup11 = [i.text for i in soup.select('p.css-1kaoc8u')]
    experiment.append(soup11)
    
    soup12 = [i.text for i in soup.select('div.css-1q7t5np > :nth-of-type(1)')]
    experiment1.append(soup12)
    
    soup13 = [i.text for i in soup.select('h3.css-1otu251')]
    år.append(soup13)
    

    


### Turning the data into DataFrame

In [3]:

df=pd.DataFrame([namn,land,pris,liter_pris,reg_nr,forpackning,storlek,alkoholhalt,
                 sockerhalt,passar_till,experiment, experiment1,år]).T
df.columns=['Namn', 'Land','Pris(SEK)', 'Literpris', 'Produktnummer','Förpackning',
            'Volym(mml)','Alkoholhalt(%)','Sockerhalt(gram/l)','passar_till','rawdata','rawdata1','Årgång']



In [4]:
df

,Namn,Land,Pris(SEK),Literpris,Produktnummer,Förpackning,Volym(mml),Alkoholhalt(%),Sockerhalt(gram/l),passar_till,rawdata,rawdata1,Årgång
0,[Brunello di Montalcino],[Italien],"[299:-, 299:-]","[Jfr.pris 398,67 kr/l, Jfr.pris 398,67 kr/l]",[Nr 95447Produktnummer: Nr 95447],[Flaska 750 ml],[750 ml],[14 %],[],"[Fyllighet, Fruktsyra, Strävhet, Lamm, Nöt, Vilt]","[Sangiovese., Mörk, röd färg med tegelnyans., ...","[RåvarorSangiovese.FärgMörk, röd färg med tege...","[Villa al Cortile, 2013]"
1,[Barolo Falletto],[Italien],"[1 949:-, 1 949:-]","[Jfr.pris 2 598,67 kr/l, Jfr.pris 2 598,67 kr/l]",[Nr 92071Produktnummer: Nr 92071],[Flaska 750 ml],[750 ml],[15 %],[],[],[Har du frågor om allergener? Chatta med oss.H...,[AllergenerHar du frågor om allergener? Chatta...,[2014]
2,[Barolo Falletto Vigna Le Roche],[Italien],"[4 299:-, 4 299:-]","[Jfr.pris 5 732,00 kr/l, Jfr.pris 5 732,00 kr/l]",[Nr 92073Produktnummer: Nr 92073],[Flaska 750 ml],[750 ml],"[14,5 %]",[],[],[Har du frågor om allergener? Chatta med oss.H...,[AllergenerHar du frågor om allergener? Chatta...,"[Riserva, 2012]"
3,[Gattinara],[Italien],"[289:-, 289:-]","[Jfr.pris 385,33 kr/l, Jfr.pris 385,33 kr/l]",[Nr 92088Produktnummer: Nr 92088],[Flaska 750 ml],[750 ml],[14 %],[],"[Fyllighet, Fruktsyra, Strävhet, Lamm, Nöt, Vilt]","[Nebbiolo., Tegelröd färg., Komplex, utvecklad...",[RåvarorNebbiolo.FärgTegelröd färg.DoftKomplex...,"[Mauro Franchino, 2013]"
4,[Bolgaré],[Bulgarien],"[69:-, 69:-]","[Jfr.pris 92,00 kr/l, Jfr.pris 92,00 kr/l]",[Nr 2110Produktnummer: Nr 2110],[Flaska 750 ml],[750 ml],"[13,5 %]",[<3 g/l],"[Fyllighet, Fruktsyra, Strävhet, Fläsk, Lamm]","[70% syrah och 30% merlot., Tät, mörk, blåröd ...","[Råvaror70% syrah och 30% merlot.FärgTät, mörk...","[Syrah Merlot, 2017]"
5,[Contino Viña del Olivo],[Spanien],"[719:-, 719:-]","[Jfr.pris 958,67 kr/l, Jfr.pris 958,67 kr/l]",[Nr 95967Produktnummer: Nr 95967],[Flaska 750 ml],[750 ml],[14 %],[],[],"[90% tempranillo och 10% graciano., Har du frå...",[Råvaror90% tempranillo och 10% graciano.Aller...,"[Viñedos del Contino, 2016]"
6,[Adobe],[Chile],"[77:-, 77:-]","[Jfr.pris 102,67 kr/l, Jfr.pris 102,67 kr/l]",[Nr 6853Produktnummer: Nr 6853],[Flaska 750 ml],[750 ml],"[13,5 %]",[<3 g/l],"[Fyllighet, Fruktsyra, Strävhet, Fläsk, Lamm, ...","[Cabernet sauvignon., Mörk, blåröd färg., Fruk...","[RåvarorCabernet sauvignon.FärgMörk, blåröd fä...","[Reserva Cabernet Sauvignon, 2018]"
7,[Marqués de Cáceres],[Spanien],"[89:-, 89:-]","[Jfr.pris 118,67 kr/l, Jfr.pris 118,67 kr/l]",[Nr 2734Produktnummer: Nr 2734],[Flaska 750 ml],[750 ml],[14 %],[<3 g/l],"[Fyllighet, Fruktsyra, Strävhet, Lamm, Nöt]",[85% tempranillo samt 15% garnacha och gracian...,[Råvaror85% tempranillo samt 15% garnacha och ...,"[Crianza, 2015]"
8,[Chateau Cantenac Brown],[Frankrike],"[675:-, 675:-]","[Jfr.pris 900,00 kr/l, Jfr.pris 900,00 kr/l]",[Nr 95514Produktnummer: Nr 95514],[Flaska 750 ml],[750 ml],"[13,5 %]",[],[],"[Cabernet sauvignon, merlot och cabernet franc...","[RåvarorCabernet sauvignon, merlot och caberne...",[2016]
9,[Masi Bonacosta],[Italien],"[65:-, 65:-]","[Jfr.pris 173,33 kr/l, Jfr.pris 173,33 kr/l]",[Nr 6998Produktnummer: Nr 6998],[Flaska 375 ml],[375 ml],[12 %],[<3 g/l],"[Fyllighet, Fruktsyra, Strävhet, Buffémat, Fis...","[Corvina, rondinella och molinara., Mörk, rödb...","[RåvarorCorvina, rondinella och molinara.FärgM...","[Valpolicella, 2018]"


In [5]:
df.to_csv('wines_red.csv')

In [6]:
df1 = pd.read_csv('wines_red.csv')
df1

,Unnamed: 0,Namn,Land,Pris(SEK),Literpris,Produktnummer,Förpackning,Volym(mml),Alkoholhalt(%),Sockerhalt(gram/l),passar_till,rawdata,rawdata1,Årgång
0,0,['Brunello di Montalcino'],['Italien'],"['299:-', '299:-']","['Jfr.pris 398,67 kr/l', 'Jfr.pris 398,67 kr/l']",['Nr 95447Produktnummer: Nr 95447'],['Flaska 750 ml'],['750 ml'],['14 %'],[],"['Fyllighet', 'Fruktsyra', 'Strävhet', 'Lamm',...","['Sangiovese.', 'Mörk, röd färg med tegelnyans...","['RåvarorSangiovese.FärgMörk, röd färg med teg...","['Villa al Cortile, 2013']"
1,1,['Barolo Falletto'],['Italien'],"['1 949:-', '1 949:-']","['Jfr.pris 2 598,67 kr/l', 'Jfr.pris 2 598,67 ...",['Nr 92071Produktnummer: Nr 92071'],['Flaska 750 ml'],['750 ml'],['15 %'],[],[],['Har du frågor om allergener? Chatta med oss....,['AllergenerHar du frågor om allergener? Chatt...,['2014']
2,2,['Barolo Falletto Vigna Le Roche'],['Italien'],"['4 299:-', '4 299:-']","['Jfr.pris 5 732,00 kr/l', 'Jfr.pris 5 732,00 ...",['Nr 92073Produktnummer: Nr 92073'],['Flaska 750 ml'],['750 ml'],"['14,5 %']",[],[],['Har du frågor om allergener? Chatta med oss....,['AllergenerHar du frågor om allergener? Chatt...,"['Riserva, 2012']"
3,3,['Gattinara'],['Italien'],"['289:-', '289:-']","['Jfr.pris 385,33 kr/l', 'Jfr.pris 385,33 kr/l']",['Nr 92088Produktnummer: Nr 92088'],['Flaska 750 ml'],['750 ml'],['14 %'],[],"['Fyllighet', 'Fruktsyra', 'Strävhet', 'Lamm',...","['Nebbiolo.', 'Tegelröd färg.', 'Komplex, utve...",['RåvarorNebbiolo.FärgTegelröd färg.DoftKomple...,"['Mauro Franchino, 2013']"
4,4,['Bolgaré'],['Bulgarien'],"['69:-', '69:-']","['Jfr.pris 92,00 kr/l', 'Jfr.pris 92,00 kr/l']",['Nr 2110Produktnummer: Nr 2110'],['Flaska 750 ml'],['750 ml'],"['13,5 %']",['<3 g/l'],"['Fyllighet', 'Fruktsyra', 'Strävhet', 'Fläsk'...","['70% syrah och 30% merlot.', 'Tät, mörk, blår...","['Råvaror70% syrah och 30% merlot.FärgTät, mör...","['Syrah Merlot, 2017']"
5,5,['Contino Viña del Olivo'],['Spanien'],"['719:-', '719:-']","['Jfr.pris 958,67 kr/l', 'Jfr.pris 958,67 kr/l']",['Nr 95967Produktnummer: Nr 95967'],['Flaska 750 ml'],['750 ml'],['14 %'],[],[],"['90% tempranillo och 10% graciano.', 'Har du ...",['Råvaror90% tempranillo och 10% graciano.Alle...,"['Viñedos del Contino, 2016']"
6,6,['Adobe'],['Chile'],"['77:-', '77:-']","['Jfr.pris 102,67 kr/l', 'Jfr.pris 102,67 kr/l']",['Nr 6853Produktnummer: Nr 6853'],['Flaska 750 ml'],['750 ml'],"['13,5 %']",['<3 g/l'],"['Fyllighet', 'Fruktsyra', 'Strävhet', 'Fläsk'...","['Cabernet sauvignon.', 'Mörk, blåröd färg.', ...","['RåvarorCabernet sauvignon.FärgMörk, blåröd f...","['Reserva Cabernet Sauvignon, 2018']"
7,7,['Marqués de Cáceres'],['Spanien'],"['89:-', '89:-']","['Jfr.pris 118,67 kr/l', 'Jfr.pris 118,67 kr/l']",['Nr 2734Produktnummer: Nr 2734'],['Flaska 750 ml'],['750 ml'],['14 %'],['<3 g/l'],"['Fyllighet', 'Fruktsyra', 'Strävhet', 'Lamm',...",['85% tempranillo samt 15% garnacha och gracia...,['Råvaror85% tempranillo samt 15% garnacha och...,"['Crianza, 2015']"
8,8,['Chateau Cantenac Brown'],['Frankrike'],"['675:-', '675:-']","['Jfr.pris 900,00 kr/l', 'Jfr.pris 900,00 kr/l']",['Nr 95514Produktnummer: Nr 95514'],['Flaska 750 ml'],['750 ml'],"['13,5 %']",[],[],"['Cabernet sauvignon, merlot och cabernet fran...","['RåvarorCabernet sauvignon, merlot och cabern...",['2016']
9,9,['Masi Bonacosta'],['Italien'],"['65:-', '65:-']","['Jfr.pris 173,33 kr/l', 'Jfr.pris 173,33 kr/l']",['Nr 6998Produktnummer: Nr 6998'],['Flaska 375 ml'],['375 ml'],['12 %'],['<3 g/l'],"['Fyllighet', 'Fruktsyra', 'Strävhet', 'Buffém...","['Corvina, rondinella och molinara.', 'Mörk, r...","['RåvarorCorvina, rondinella och molinara.Färg...","['Valpolicella, 2018']"
